# Getting started `musif`

`musif` is a Python library to analyze music scores. It is a tool to massively extract features from MusicXML and MuseScore files.

`musif` was born in the context of the [ERC Project "DIDONE"](https://didone.eu/) and, consequently,
it is specialized in 18th-century Italian opera arias. However, it is also prepared to work with other repertoires.

This tutorial is an introduction for people who are not experts in programming. If you are already an expert, just skip to the [Data Section](#data) and then go to the [Advanced Tutorial](https://musif.didone.eu/Tutorial%20poprock.html).


## Installation

First, you should install [`Python`](https://www.python.org/downloads/) > 3.10. An easy way to do this is by using [`Anaconda`](https://www.anaconda.com/products/distribution), especially if you are not used to commandline interface.A
Once you have installed `anaconda`:
1. Launch the `anaconda-navigator`
2. [Create an environment](https://docs.anaconda.com/navigator/getting-started/#managing-environments) selecting python version >= 3.10
3. Switch to the newly created environment by clicking on its name
4. [Install `jupyter`](https://docs.anaconda.com/anaconda/user-guide/getting-started/#run-python-in-a-jupyter-notebook).

To install `musif`:
1. [Download](https://raw.githubusercontent.com/DIDONEproject/musif/main/docs/source/Tutorial.ipynb) this notebook.
2. Start `jupyter` in your Anaconda environment.
3. Open this tutorial.
4. Run the following cell by clicking on it and pressing Ctrl+Enter.

In [1]:
!pip install musif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.6 MB/s eta 0:00:00 MB/s eta 0:00:01


Here, the `!` is a special command that executes commands in the terminal. After having run it, you may need to restart the notebook (click the circular arrow ↻ in the top bar, near the icons ▶ and ⏹)

Now, let's import `musif`:

## Introduction

If you are new to Python, we suggest you to read an introductory tutorial for it, for instance, [this one](https://www.w3schools.com/python/default.asp). 

In the following, we will introduce some technical terminology that may be useful to you to understand technical documentation while working with `musif`:

* A _function_ is a way to represent code that is convenient for humans. You can think of functions as mathematical functions, with some input and some output. However, some programming languages call them _procedures_; this is not the case with Python, but this name allows grasping what functions are, after all: successions of commands that the computer has to execute.

* An _object_ is a computational way to represent information _and_ code in the memory of computers; you can think of objects as real concepts of the real world: objects have properties (in Python named _fields_) and functionalities (named _methods_). For instance, an object could be a vehicle, which has some properties (length, maximum speed, number of wheels) and some functionalities (accelerate, decelerate, stop). Objects can also have specializations (named _children_): in our example, a _child_ of vehicle could be the car and another _child_ could be the bike: they have different properties and apply the functionalities in a different way. Both the vehicle, the car, and the bike may have instances: the car that you use everyday to go to work is different from your friend's even if they have the same exact properties, because they are two different concrete objects. Technically, those two cars are two _instances_ of the same _class_. To create an instance, you have to call a function, generically named _constructor_, which takes as arguments the class and the other properties. This function will return the instance. To use `musif`, you don't need to know a lot about objects, but while you search the web it is good to have a little of knowledge.

* A _DataFrame_ is another way to represent information for computers. They are designed to be extremely efficient, even if sometimes some aspects of the information can get lost. They are mainly used for data science problems. You can think of a _DataFrame_ as a table, with rows and columns. Usually, rows are _instances_ while columns are _properties_. In data science, these words often become _samples_ and _features_/_variables_. A typical operation is to select only certain columns (properties) or only certain rows (instances) to select subset of the data or to modify the data itself.

* Don't be scared to use web search engines such as Google: searching the web in a proper way is one of the most important skills a programmer has!

### Main objects

When using `musif`, you will usually interface with two objects:
1. [`FeaturesExtractor()`](API/musif.extract.html#musif.extract.extract.FeaturesExtractor), which reads music scores and computes a DataFrame containing all the extracted features. In the simplest case, each row represents a music score, while each column represents a feature.
2. [`DataProcessor()`](API/musif.process.html#musif.process.processor.DataProcessor), which takes the DataFrame with all the features in it and post-processes it to clean, improve, and possibly modify some of the features.

These two objects take as input two different configurations that modify their behavior. In other words, the constructors of `FeaturesExtractor` and `DataProcessor` can accept a wide range of arguments.

But let's proceed step by step!

In [ ]:
import urllib.request
import zipfile
from pathlib import Path

data_dir = Path("data")
dataset_path = "dataset.zip"
urllib.request.urlretrieve("https://zenodo.org/record/4027957/files/AnatomyComposerAttributionMIDIFilesAndFeatureData_1_0.zip?download=1", dataset_path)
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

## Configuration

Let's create a configuration for our experiment. Configurations can be expressed using a `yaml` file or with key-value arguments. `yaml` files are designed for complex projects, while key-value arguments are perfect for simple situations like this.

Key-value arguments are something similar to a dictionary: There is a _key_ which must be unique in the dictionary; each _key_ is associated with a _value_, which can instead be repeated. Python can retrieve a value using its key in  a very efficient way!

First, we'll need to import the class that describes how a configuration is:

Now, we can call its constructor to obtain a configuration object:

In [ ]:
import glob

config = ExtractConfiguration(
    None,
    musescore_dir="data",
    basic_modules=["scoring"],
    features=["core", "ambitus", "interval", "tempo", 
              "density", "texture", "lyrics", "scale", 
              "key", "dynamics", "rhytm"],
)

## Feature extraction

Now that we have our configuration, we pass it to the function that creates `FeaturesExtraction` objects. This function is exactly named `FeaturesExtraction`:

In [ ]:
from musif.extract.extract import FeaturesExtractor

extractor = FeaturesExtractor(config)

In [ ]:
extractor = FeaturesExtractor(
    None,
    musescore_dir="data",
    basic_modules=["scoring"],
    features=["core", "ambitus", "melody", "tempo", 
              "density", "texture", "lyrics", "scale", 
              "key", "dynamics", "rhythm"]
)

Before starting the extraction, we also need to tell MuseScore the type of files it should look for. In this case, we want it to look for files with extension `'.mid'`. By default, it would look for `.mscx` files, so we need to change it:

Now, we can start the extraction using the method `extract`. It will return a `DataFrame`:

## Post-processing

Most of the features we have computed actually need some post-processing, for instance to replace `NaN` with 0, merge columns, or remove features created while computing other features.

For this, we need a further step. In the next cell we will:
1. Instantiate a `DataProcessor` object using:
    * the generated DataFrame
    * the default configuration (i.e. `None` in place of the yaml file/configuration object)
2. Call the method `process()` of that object to start the post-processing of the features
3. We retrieve the post-processed data from the field `data`
4. We print the size of the DataFrame.

In [ ]:
from musif.process.processor import DataProcessor

processed_df = DataProcessor(df, None).process().data

# with `.shape` you can see the number of rows and columns of the dataframe
processed_df.shape

In [ ]:
processed_df.dropna(axis=1, inplace=True)
processed_df.shape

The dataset was not very well formatted and this is the reason why only a few features remain.

## Statistical processing

Let's try to classify the features. We will setup a feature-learning approach with an autoencoder architecture.

For this, we will use `sklearn` and its Multilayer Perceptron, so you will need to [install](https://docs.anaconda.com/navigator/getting-started/#managing-packages) `scikit-learn` and `seaborn` packages in your anaconda environment.

In the next cell, the topic becomes a little more technical, but it's just an example to show that you can use this DataFrame for statistical analysis. We will first remove redundant information (the `FileName` and the `Id` columns that were automatically assigned by the `FeatureExtractor`). 

Then, we will create a model which:
1. Assigns a number to each feature that has strings as values (`OrdinalEncoder`).
2. Standardizes the features to get comparable values.
3. Trains a simple feed-forward fully connected autoencoder with ReLU activations and LBGFS optimizer.

The objective is to learn a 2D space where the 396 extracted features can be represented without loosing information.

Now, we will attach a method `transform` to the MLPClassifier which returns the activations at the inner layer with 2 outputs, that we interpret as latent features. To compare the features, we will scale them in [0, 1].

Then, we plot the music scores according to the learned feature space.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mlpclassifier = model['mlpregressor']

def mytransform_method(X):
    activations = [None for _ in range(mlpclassifier.n_layers_)]
    activations[0] = X
    X = mlpclassifier._forward_pass(activations)[-6]
    return MinMaxScaler().fit_transform(X)

mlpclassifier.transform = mytransform_method

learned_features = model.transform(processed_df)

learned_features.shape

In [ ]:
from sklearn.decomposition import PCA

pca_pipeline = make_pipeline(
    preprocessor, PCA(2), MinMaxScaler()
)
data_pca = pca_pipeline.fit_transform(processed_df)
ax = seaborn.scatterplot(x=data_pca[:, 0], y=data_pca[:, 1])